## Question 1

### a

In [42]:
import pandas as pd

# Load data from CSV files
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Decision variables x[i, j] represent the tons of raw material j used in blend i.
# There are 50 blends and 100 raw materials, so there are 50 * 100 = 5000 decision variables.


#### Decision variables x[i, j] represent the tons of raw material j used in blend i.
#### There are 50 blends and 100 raw materials, so there are 50 * 100 = 5000 decision variables.

### b

In [43]:
df_blend.head()

,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,...,quality_94,quality_95,quality_96,quality_97,quality_98,quality_99,quality_100,demand,quality_min,quality_max
0,80.125450,94.259594,86.347745,66.517377,62.822750,85.696771,61.060452,83.431023,97.609210,83.018967,...,74.825686,92.511983,97.889943,99.440043,90.135127,75.050383,63.340029,112.571674,84.082659,96.420316
1,91.085877,82.336170,76.968880,96.254175,64.447899,79.705004,60.454146,78.746426,62.252131,64.752717,...,86.868027,84.725130,74.326509,64.542304,86.862928,80.812308,90.892736,354.564165,77.395619,90.841400
2,80.806540,94.087260,82.076274,82.437519,95.066144,76.139315,65.360609,61.151307,90.205490,84.812382,...,73.099894,64.790485,95.621091,83.743698,87.164093,91.566850,79.937688,225.742392,76.448949,91.616287
3,63.476812,81.484262,83.473645,89.817579,77.266382,65.103212,71.351036,74.523292,85.836690,82.831132,...,80.108816,69.288508,95.982983,75.355649,81.742114,96.258884,84.969520,303.428276,79.894528,98.985542
4,64.675922,97.593285,85.108322,73.396225,65.570883,91.761008,84.802910,81.338444,95.755703,91.543888,...,98.442813,65.946509,76.584965,63.413987,99.874970,80.087800,83.815401,463.026590,84.856505,96.064291


In [44]:
df_mat.head()

,availability,cost,p_max
0,3650.040029,4.996321,0.593264
1,4465.879557,9.605714,0.698169
2,8660.230044,7.855952,0.415004
3,3852.298046,6.789268,0.515288
4,2525.434720,3.248149,0.513329


In [45]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Define sets based on indices
raw_materials = df_mat.index.tolist()  # j = 0..99
blends = df_blend.index.tolist()       # i = 0..49

# Extract columns from blending_materials.csv
availability = df_mat['availability'].to_dict()  # A_j: available tons for each raw material j
cost = df_mat['cost'].to_dict()                  # cost per ton for each raw material j
pmax = df_mat['p_max'].to_dict()                 # p_j^max: maximum proportion for each raw material j

# Extract columns from blending_blends.csv
demand = df_blend['demand'].to_dict()            # D_i: demand for each blend i
Qmin = df_blend['quality_min'].to_dict()         # Q_i^min: minimum quality for each blend i
Qmax = df_blend['quality_max'].to_dict()         # Q_i^max: maximum quality for each blend i

# Build quality contribution dictionary: quality[i][j] = q_ij
# Assume quality columns are named 'quality_1', 'quality_2', ..., 'quality_100'
quality = {}
for i in blends:
    quality[i] = {}
    for j in raw_materials:
        col_name = f'quality_{j+1}'
        quality[i][j] = df_blend.loc[i, col_name]

# Create the Gurobi model
model = gp.Model("EcoClean_Blending")

# Decision variables: x[i,j] = tons of raw material j used in blend i
# Non-negativity is enforced by lb=0
x = model.addVars(blends, raw_materials, lb=0, name="x")

# ---------------------------
# 1) Demand Constraints:
# For each blend i: sum_{j=1}^{100} x[i,j] == D_i
for i in blends:
    model.addConstr(
        gp.quicksum(x[i, j] for j in raw_materials) == demand[i],
        name=f"demand_{i}"
    )

# ---------------------------
# 2) Quality Constraints:
# For each blend i: Qmin[i]*D_i <= sum_{j=1}^{100} (q[i,j]*x[i,j]) <= Qmax[i]*D_i
for i in blends:
    model.addConstr(
        gp.quicksum(quality[i][j] * x[i, j] for j in raw_materials) >= Qmin[i] * demand[i],
        name=f"qualityMin_{i}"
    )
    model.addConstr(
        gp.quicksum(quality[i][j] * x[i, j] for j in raw_materials) <= Qmax[i] * demand[i],
        name=f"qualityMax_{i}"
    )

# ---------------------------
# 3) Raw Material Availability Constraints:
# For each raw material j: sum_{i=1}^{50} x[i,j] <= A_j
for j in raw_materials:
    model.addConstr(
        gp.quicksum(x[i, j] for i in blends) <= availability[j],
        name=f"availability_{j}"
    )

# ---------------------------
# 4) Proportion (Operational) Constraints:
# For each blend i and each raw material j: x[i,j] <= pmax[j] * D_i
for i in blends:
    for j in raw_materials:
        model.addConstr(
            x[i, j] <= pmax[j] * demand[i],
            name=f"proportion_{i}_{j}"
        )

# ---------------------------
# 5) Non-Negativity Constraints (explicitly added)
# For each blend i and raw material j: x[i,j] >= 0
for i in blends:
    for j in raw_materials:
        model.addConstr(
            x[i, j] >= 0,
            name=f"nonneg_{i}_{j}"
        )

# Set the objective: minimize total production cost
model.setObjective(
    gp.quicksum(cost[j] * x[i, j] for i in blends for j in raw_materials),
    GRB.MINIMIZE
)

# Optimize the model
model.optimize()

# If the model is optimal, print the minimum production cost
if model.status == GRB.OPTIMAL:
    print("Minimum production cost:", model.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10250 rows, 5000 columns and 30000 nonzeros
Model fingerprint: 0x1da20f82
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 10081 rows and 0 columns
Presolve time: 0.00s
Presolved: 169 rows, 5050 columns, 13500 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.950e+03
 Factor NZ  : 1.030e+04 (roughly 2 MB of memory)
 Factor Ops : 7.503e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.01 seconds (0.01 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iterati

#### The minimum production cost associated with this linear program is 30879.62

### c

In [46]:
# --- Sensitivity Analysis for raw material j=5 (index 5) ---
# Reduce cost for raw material j=5 by $1.00 per ton
original_cost_j5 = cost[5]
new_cost_j5 = original_cost_j5 - 1.0

# Update the objective: use new_cost_j5 for j=5 and original cost for others
new_obj = gp.quicksum((new_cost_j5 if j == 5 else cost[j]) * x[i, j]
                      for i in blends for j in raw_materials)
model.setObjective(new_obj, GRB.MINIMIZE)

# Re-optimize the model
model.optimize()

# Print the usage of raw material j=5 in each blend to see if it is included
print("Usage of raw material j=5 in each blend after price reduction:")
for i in blends:
    print(f"Blend {i}: x[{i},5] =", x[i, 5].X)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10250 rows, 5000 columns and 30000 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.9924477e+28   2.905252e+31   5.992448e-02      0s
      22    3.0798916e+04   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.01 seconds (0.01 work units)
Optimal objective  3.079891604e+04
Usage of raw material j=5 in each blend after price reduction:
Blend 0: x[0,5] = 52.037168689435326
Blend 1: x[1,5] = 0.0
Blend 2: x[2,5] = 0.0
Blend 3: x[3,5] = 0.0
Blend 4: x[4,5] = 214.03779329403895
Blend 5: x[5,5] = 22.082373799646636
Blend 6: x[6,5] = 0.0
Blend 7: x[7,5] =

#### Yes, the reduced price causes raw material j=5 to be included in the blend formulations. In our original model, the cost coefficients led to an optimal solution that did not favor using material j=5 in many blends. After decreasing its price by $1.00 per ton, the sensitivity analysis shows nonzero usage in several blends—for example, Blend 0 uses ~52.04 tons, Blend 4 ~214.04 tons, and Blend 5 ~22.08 tons. This change in usage indicates that the lower cost made raw material j=5 more attractive in minimizing total production cost, and thus it is now included in the optimal blend mix.

### d

#### The quality constraint requires that:

  ∑₍j=1₎¹⁰⁰ qᵢⱼ·xᵢⱼ ≥ Qminᵢ·Dᵢ.

#### If Qminᵢ increases by 1, the required total quality for each blend increases by Dᵢ. If the allowable increase in the sensitivity report for Qminᵢ is greater than 1, then the optimal basis (and solution) remains unchanged. In other words, provided the allowable increase exceeds 1 unit for each blend, raising Qminᵢ by one unit will not alter the optimal solution.

### e

In [47]:
# --- Increase availability for raw material j=73 by 20 tons and re-optimize ---
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Define sets
raw_materials = df_mat.index.tolist()  # j = 0..99
blends = df_blend.index.tolist()       # i = 0..49

# Extract parameters from blending_materials.csv
availability = df_mat['availability'].to_dict()   # A_j
cost = df_mat['cost'].to_dict()                   # cost per ton
pmax = df_mat['p_max'].to_dict()                  # p_j^max

# Extract parameters from blending_blends.csv
demand = df_blend['demand'].to_dict()              # D_i
Qmin = df_blend['quality_min'].to_dict()           # Q_i^min
Qmax = df_blend['quality_max'].to_dict()           # Q_i^max

# Build quality contribution dictionary: quality[i][j] = q_ij 
# (assuming columns 'quality_1' ... 'quality_100')
quality = {}
for i in blends:
    quality[i] = {}
    for j in raw_materials:
        col_name = f'quality_{j+1}'
        quality[i][j] = df_blend.loc[i, col_name]

# Create the Gurobi model
model = gp.Model("EcoClean_Blending")

# Decision variables: x[i,j] = tons of raw material j used in blend i
# Non-negativity is enforced by lb=0, but we also add explicit constraints below.
x = model.addVars(blends, raw_materials, lb=0, name="x")

# ---------------------------
# 1) Demand Constraints:
# For each blend i: sum_{j=1}^{100} x[i,j] == D_i
for i in blends:
    model.addConstr(
        gp.quicksum(x[i, j] for j in raw_materials) == demand[i],
        name=f"demand_{i}"
    )

# ---------------------------
# 2) Quality Constraints:
# For each blend i: Qmin[i]*D_i <= sum_{j=1}^{100} (q[i,j]*x[i,j]) <= Qmax[i]*D_i
for i in blends:
    model.addConstr(
        gp.quicksum(quality[i][j] * x[i, j] for j in raw_materials) >= Qmin[i] * demand[i],
        name=f"qualityMin_{i}"
    )
    model.addConstr(
        gp.quicksum(quality[i][j] * x[i, j] for j in raw_materials) <= Qmax[i] * demand[i],
        name=f"qualityMax_{i}"
    )

# ---------------------------
# 3) Raw Material Availability Constraints:
# For each raw material j: sum_{i=1}^{50} x[i,j] <= A_j
availability_constrs = {}
for j in raw_materials:
    availability_constrs[j] = model.addConstr(
        gp.quicksum(x[i, j] for i in blends) <= availability[j],
        name=f"availability_{j}"
    )

# ---------------------------
# 4) Proportion (Operational) Constraints:
# For each blend i and raw material j: x[i,j] <= pmax[j] * D_i
for i in blends:
    for j in raw_materials:
        model.addConstr(
            x[i, j] <= pmax[j] * demand[i],
            name=f"proportion_{i}_{j}"
        )

# ---------------------------
# 5) Non-Negativity Constraints (explicit)
# For each blend i and raw material j: x[i,j] >= 0
for i in blends:
    for j in raw_materials:
        model.addConstr(
            x[i, j] >= 0,
            name=f"nonneg_{i}_{j}"
        )

# Set the objective: minimize total production cost
model.setObjective(
    gp.quicksum(cost[j] * x[i, j] for i in blends for j in raw_materials),
    GRB.MINIMIZE
)

# Optimize the original model
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Original minimum production cost:", model.objVal)

# --- Now, modify the availability for raw material j=73 by adding 20 tons ---
j_update = 73
new_availability_j73 = availability[j_update] + 20

# Update the RHS for the availability constraint of raw material j=73
availability_constrs[j_update].setAttr(GRB.Attr.RHS, new_availability_j73)

# Re-optimize the model with the new availability
model.optimize()

if model.status == GRB.OPTIMAL:
    print("New minimum production cost:", model.objVal)
    
    # Retrieve the dual value (shadow price) for the availability constraint of raw material j=73
    dual_j73 = availability_constrs[j_update].Pi
    print("Dual value (shadow price) for raw material 73:", dual_j73)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10250 rows, 5000 columns and 30000 nonzeros
Model fingerprint: 0x1da20f82
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 10081 rows and 0 columns
Presolve time: 0.00s
Presolved: 169 rows, 5050 columns, 13500 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.950e+03
 Factor NZ  : 1.030e+04 (roughly 2 MB of memory)
 Factor Ops : 7.503e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.01 seconds (0.01 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iterati

In [48]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Define sets based on indices
raw_materials = df_mat.index.tolist()  # 0 to 99 for raw materials
blends = df_blend.index.tolist()       # 0 to 49 for blends

# Extract parameters from blending_materials.csv
availability = df_mat['availability'].to_dict()  # A_j
cost = df_mat['cost'].to_dict()                  # cost per ton for each raw material j
pmax = df_mat['p_max'].to_dict()                 # p_j^max

# Extract parameters from blending_blends.csv
demand = df_blend['demand'].to_dict()            # D_i
Qmin = df_blend['quality_min'].to_dict()         # Q_i^min
Qmax = df_blend['quality_max'].to_dict()         # Q_i^max

# Build quality contribution dictionary: quality[i][j] = q_ij
# Assumes columns are named 'quality_1', 'quality_2', ..., 'quality_100'
quality = {}
for i in blends:
    quality[i] = {}
    for j in raw_materials:
        col_name = f'quality_{j+1}'
        quality[i][j] = df_blend.loc[i, col_name]

# Create a new Gurobi model for part (e)
model_e = gp.Model("EcoClean_Blending_Extra")

# Decision variables:
# For raw materials j != 73, define x_e[i,j]
x_e = {}
for i in blends:
    for j in raw_materials:
        if j != 73:
            x_e[i, j] = model_e.addVar(lb=0, name=f"x_{i}_{j}")

# For raw material j = 73, split usage into:
# x_e_73[i] : usage from normal supply (cost = cost[73])
# y[i]     : usage from extra supply (cost = p_extra, to be determined)
x_e_73 = {}
y = {}
for i in blends:
    x_e_73[i] = model_e.addVar(lb=0, name=f"x_{i}_73")
    y[i] = model_e.addVar(lb=0, name=f"y_{i}")

model_e.update()

# Parameter: extra cost per ton for additional supply of raw material 73
p_extra = 0  # Set this value as needed

# Objective:
# For j != 73: cost[j]*x_e[i,j]
# For j = 73: cost[73]*x_e_73[i] + p_extra*y[i]
obj_expr = gp.quicksum(cost[j] * x_e[i, j] for i in blends for j in raw_materials if j != 73) \
    + gp.quicksum(cost[73] * x_e_73[i] for i in blends) \
    + gp.quicksum(p_extra * y[i] for i in blends)

model_e.setObjective(obj_expr, GRB.MINIMIZE)

# --- Constraint 1: Demand Constraints ---
# For each blend i: sum_{j !=73} x_e[i,j] + (x_e_73[i] + y[i]) == D_i
for i in blends:
    expr = gp.quicksum(x_e[i, j] for j in raw_materials if j != 73) + (x_e_73[i] + y[i])
    model_e.addConstr(expr == demand[i], name=f"demand_{i}")

# --- Constraint 2: Quality Constraints ---
# For each blend i:
# Lower: sum_{j !=73} quality[i][j]*x_e[i,j] + quality[i][73]*(x_e_73[i] + y[i]) >= Qmin[i]*D_i
# Upper: similarly <= Qmax[i]*D_i
for i in blends:
    expr_quality = gp.quicksum(quality[i][j] * x_e[i, j] for j in raw_materials if j != 73) \
        + quality[i][73] * (x_e_73[i] + y[i])
    model_e.addConstr(expr_quality >= Qmin[i] * demand[i], name=f"qualityMin_{i}")
    model_e.addConstr(expr_quality <= Qmax[i] * demand[i], name=f"qualityMax_{i}")

# --- Constraint 3: Raw Material Availability Constraints ---
# For j != 73: total usage <= A_j
for j in raw_materials:
    if j != 73:
        model_e.addConstr(gp.quicksum(x_e[i, j] for i in blends) <= availability[j], name=f"avail_{j}")
    else:
        # For raw material 73, normal supply: sum_{i} x_e_73[i] <= availability[73]
        model_e.addConstr(gp.quicksum(x_e_73[i] for i in blends) <= availability[73], name=f"avail_{73}")

# --- Constraint 4: Proportion (Operational) Constraints ---
# For each blend i and j != 73: x_e[i,j] <= pmax[j]*D_i
# For j = 73: (x_e_73[i] + y[i]) <= pmax[73]*D_i
for i in blends:
    for j in raw_materials:
        if j != 73:
            model_e.addConstr(x_e[i, j] <= pmax[j] * demand[i], name=f"prop_{i}_{j}")
        else:
            model_e.addConstr((x_e_73[i] + y[i]) <= pmax[73] * demand[i], name=f"prop_{i}_73")

# --- Constraint 5: Non-Negativity Constraints ---
# These are already enforced by variable bounds, but we add them explicitly.
for i in blends:
    for j in raw_materials:
        if j != 73:
            model_e.addConstr(x_e[i, j] >= 0, name=f"nonneg_{i}_{j}")
    model_e.addConstr(x_e_73[i] >= 0, name=f"nonneg_{i}_73")
    model_e.addConstr(y[i] >= 0, name=f"nonneg_y_{i}")

# --- Additional Constraint for Extra Supply ---
# The total extra supply for raw material 73 cannot exceed 20 tons.
model_e.addConstr(gp.quicksum(y[i] for i in blends) <= 20, name="extra_supply")

model_e.update()

# Optimize the extended model
model_e.optimize()

# After optimization, print total extra usage for raw material 73 and the new objective value
if model_e.status == GRB.OPTIMAL:
    total_extra_used = sum(y[i].X for i in blends)
    print("Total extra supply used for raw material 73:", total_extra_used)
    print("New objective value:", model_e.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10301 rows, 5050 columns and 30300 nonzeros
Model fingerprint: 0x491eca58
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+04]
Presolve removed 10081 rows and 0 columns
Presolve time: 0.00s
Presolved: 220 rows, 5100 columns, 13750 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.250e+03
 Factor NZ  : 1.081e+04 (roughly 2 MB of memory)
 Factor Ops : 7.879e+05 (less than 1 second per iteration)
 Threads    : 1

Barrier performed 0 iterations in 0.01 seconds (0.01 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Iterati

#### The baseline model yields a cost of about \$30,879.62. With 20 extra tons of raw material 73 (at zero extra cost), the cost drops to approximately \$30,831.47—a saving of \$48.15 in total. Dividing \$48.15 by 20 tons gives about \$2.41 per ton. Thus, EcoGreen can pay up to roughly \$2.41 per additional ton for raw material 73 and still achieve a net cost reduction.

### f

In [49]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Define sets
raw_materials = df_mat.index.tolist()  # j = 0,...,99
blends = df_blend.index.tolist()       # i = 0,...,49

# Extract parameters from blending_materials.csv
availability = df_mat['availability'].to_dict()  # A_j
cost = df_mat['cost'].to_dict()                  # cost per ton for each raw material j
pmax = df_mat['p_max'].to_dict()                 # p_j^max

# Extract parameters from blending_blends.csv
demand = df_blend['demand'].to_dict()            # D_i
Qmin = df_blend['quality_min'].to_dict()         # Q_i^min
Qmax = df_blend['quality_max'].to_dict()         # Q_i^max

# Build quality contribution dictionary: quality[i][j] = q_ij
# Assumes columns are named 'quality_1', 'quality_2', ..., 'quality_100'
quality = {}
for i in blends:
    quality[i] = {}
    for j in raw_materials:
        col_name = f'quality_{j+1}'
        quality[i][j] = df_blend.loc[i, col_name]

# Create a new Gurobi model for the nonlinear program (part f)
model_f = gp.Model("EcoClean_Nonlinear_Operational")

# Decision variables: x[i,j] = tons of raw material j used in blend i
# Non-negativity is enforced by lb=0
x = model_f.addVars(blends, raw_materials, lb=0, name="x")

# Objective: minimize total production cost
model_f.setObjective(
    gp.quicksum(cost[j] * x[i,j] for i in blends for j in raw_materials),
    GRB.MINIMIZE
)

# 1) Demand Constraints:
# For each blend i: sum_{j=1}^{100} x[i,j] == D_i
for i in blends:
    model_f.addConstr(
        gp.quicksum(x[i,j] for j in raw_materials) == demand[i],
        name=f"demand_{i}"
    )

# 2) Quality Constraints:
# For each blend i: Qmin[i]*D_i <= sum_{j=1}^{100} (q[i,j]*x[i,j]) <= Qmax[i]*D_i
for i in blends:
    model_f.addConstr(
        gp.quicksum(quality[i][j] * x[i,j] for j in raw_materials) >= Qmin[i] * demand[i],
        name=f"qualityMin_{i}"
    )
    model_f.addConstr(
        gp.quicksum(quality[i][j] * x[i,j] for j in raw_materials) <= Qmax[i] * demand[i],
        name=f"qualityMax_{i}"
    )

# 3) Raw Material Availability Constraints:
# For each raw material j: sum_{i=1}^{50} x[i,j] <= A_j
for j in raw_materials:
    model_f.addConstr(
        gp.quicksum(x[i,j] for i in blends) <= availability[j],
        name=f"avail_{j}"
    )

# 4) Updated Operational (Proportion) Constraints:
# For each blend i and each raw material j:
# x[i,j] <= pmax[j] * (sum_{k=1}^{100} (x[i,k])^2)
for i in blends:
    # Define the sum of squares for blend i
    sq_sum = gp.quicksum(x[i,k]*x[i,k] for k in raw_materials)
    for j in raw_materials:
        model_f.addQConstr(
            x[i,j] <= pmax[j] * sq_sum,
            name=f"prop_{i}_{j}"
        )

# 5) Non-Negativity Constraints (explicitly added)
for i in blends:
    for j in raw_materials:
        model_f.addConstr(
            x[i,j] >= 0,
            name=f"nonneg_{i}_{j}"
        )

model_f.update()
model_f.optimize()

if model_f.status == GRB.OPTIMAL:
    print("Minimum production cost (nonlinear operational constraint):", model_f.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5250 rows, 5000 columns and 25000 nonzeros
Model fingerprint: 0x4d5c4ffd
Model has 5000 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  QMatrix range    [3e-01, 7e-01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 5e+04]
Presolve removed 5050 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 0 rows and 50 columns
Presolve removed 5 rows and 0 columns
Presolve time: 0.04s
Presolved: 10295 rows, 10051 columns, 44455 nonzeros
Presolved model has 5000 bilinear constraint(s)
Variable types: 10051 continuous, 0 integer (0 binary)

Root relaxation: objective 3.084751e+04, 1815 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Cur

#### The minimum production cost for the nonlinear program is $30,847.51

### g

In [50]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Load data
df_mat = pd.read_csv('/Users/Sam/Downloads/blending_materials.csv')
df_blend = pd.read_csv('/Users/Sam/Downloads/blending_blends.csv')

# Define sets based on indices
raw_materials = df_mat.index.tolist()  # j = 0,...,99
blends = df_blend.index.tolist()       # i = 0,...,49

# Extract parameters from blending_materials.csv
availability = df_mat['availability'].to_dict()  # A_j
cost = df_mat['cost'].to_dict()                  # cost per ton for each raw material j
pmax = df_mat['p_max'].to_dict()                 # p_j^max

# Extract parameters from blending_blends.csv
demand = df_blend['demand'].to_dict()            # D_i
Qmin = df_blend['quality_min'].to_dict()         # Q_i^min
Qmax = df_blend['quality_max'].to_dict()         # Q_i^max

# Build quality contribution dictionary: quality[i][j] = q_ij
quality = {}
for i in blends:
    quality[i] = {}
    for j in raw_materials:
        col_name = f'quality_{j+1}'
        quality[i][j] = df_blend.loc[i, col_name]

# Create a new Gurobi model for part (g)
model_g = gp.Model("EcoClean_Squared_Costs")

# Decision variables: x[i,j] = tons of raw material j used in blend i
x = model_g.addVars(blends, raw_materials, lb=0, name="x")

# Objective: minimize the sum of squared production costs,
# i.e., minimize sum_{i,j} (cost[j]*x[i,j])^2
model_g.setObjective(
    gp.quicksum((cost[j] * x[i,j]) * (cost[j] * x[i,j]) for i in blends for j in raw_materials),
    GRB.MINIMIZE
)

# 1) Demand Constraints: for each blend i, sum_{j} x[i,j] == D_i
for i in blends:
    model_g.addConstr(
        gp.quicksum(x[i,j] for j in raw_materials) == demand[i],
        name=f"demand_{i}"
    )

# 2) Quality Constraints: for each blend i,
#    Qmin[i]*D_i <= sum_{j} (q[i,j]*x[i,j]) <= Qmax[i]*D_i
for i in blends:
    model_g.addConstr(
        gp.quicksum(quality[i][j] * x[i,j] for j in raw_materials) >= Qmin[i] * demand[i],
        name=f"qualityMin_{i}"
    )
    model_g.addConstr(
        gp.quicksum(quality[i][j] * x[i,j] for j in raw_materials) <= Qmax[i] * demand[i],
        name=f"qualityMax_{i}"
    )

# 3) Raw Material Availability Constraints: for each raw material j,
#    sum_{i} x[i,j] <= A_j
for j in raw_materials:
    model_g.addConstr(
        gp.quicksum(x[i,j] for i in blends) <= availability[j],
        name=f"availability_{j}"
    )

# 4) Proportion (Operational) Constraints: for each blend i and raw material j,
#    x[i,j] <= pmax[j] * D_i
for i in blends:
    for j in raw_materials:
        model_g.addConstr(
            x[i,j] <= pmax[j] * demand[i],
            name=f"prop_{i}_{j}"
        )

# 5) Non-Negativity Constraints: (explicitly, though enforced by lb=0)
for i in blends:
    for j in raw_materials:
        model_g.addConstr(
            x[i,j] >= 0,
            name=f"nonneg_{i}_{j}"
        )

model_g.update()
model_g.optimize()

if model_g.status == GRB.OPTIMAL:
    print("Optimal objective (sum of squared production costs):", model_g.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10250 rows, 5000 columns and 30000 nonzeros
Model fingerprint: 0xe482f004
Model has 5000 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 5e+04]
Presolve removed 10081 rows and 0 columns
Presolve time: 0.00s
Presolved: 169 rows, 5050 columns, 13500 nonzeros
Presolved model has 5000 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.950e+03
 Factor NZ  : 1.030e+04 (roughly 2 MB of memory)
 Factor Ops : 7.503e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  

#### The optimal value for the objective function in this case is 911296.84

### h 

In [51]:
# After model_f.optimize(), count the nonzero decision variables in part (f)
tol = 1e-6
nonzero_count = sum(1 for i in blends for j in raw_materials if x[i, j].X > tol)
print("Number of nonzero decision variables in part (f):", nonzero_count)


Number of nonzero decision variables in part (f): 5000


In [52]:
# After model_g.optimize(), count the nonzero decision variables in part (g)
tol = 1e-6
nonzero_count_g = sum(1 for i in blends for j in raw_materials if x[i, j].X > tol)
print("Number of nonzero decision variables in part (g):", nonzero_count_g)


Number of nonzero decision variables in part (g): 5000


#### Both models result in exactly 5,000 nonzero decision variables. This happens because the structure of the blending problem forces every blend to meet its exact demand and satisfy the quality and proportion constraints. In both the nonlinear operational constraint (part f) and the squared cost objective (part g), the L₂-type formulations do not promote sparsity—instead, they produce smooth solutions where every raw material contributes a positive amount, yielding a dense solution with all 5,000 variables nonzero.

### i 

#### In part (g), the objective is to minimize the sum of squared production costs (an L₂-type penalty). Unlike L₁ regularization (as in Lasso), which can drive coefficients exactly to zero, the L₂ penalty (used in ridge regression) only shrinks coefficients without setting them to zero. Moreover, the linear constraints of the blending problem force the solution to meet exact demand and quality targets, resulting in every decision variable taking a positive value. This is why, even though ridge regression is sometimes thought to encourage sparsity, in our formulation the squared cost objective leads to a dense solution where all 5,000 variables are nonzero.

## Question 2

### a

In [53]:
df_speed = pd.read_csv('/Users/Sam/Downloads/welders_speed_data.csv')
df_data = pd.read_csv('/Users/Sam/Downloads/welders_data.csv')

In [54]:
df_speed.head()

,Unnamed: 0,Welder_1,Welder_2,Welder_3,Welder_4,Welder_5,Welder_6,Welder_7,Welder_8,Welder_9,...,Welder_135,Welder_136,Welder_137,Welder_138,Welder_139,Welder_140,Welder_141,Welder_142,Welder_143,Welder_144
0,Welder_1,4.0,3.5,3.0,3.0,3.0,4.0,4.0,4.0,3.0,...,2.5,3.0,3.0,4.0,3.5,4.0,2.5,3.0,4.0,2.5
1,Welder_2,3.5,3.0,2.5,2.5,2.5,3.5,3.5,3.5,2.5,...,2.0,2.5,2.5,3.5,3.0,3.5,2.0,2.5,3.5,2.0
2,Welder_3,3.0,2.5,2.0,2.0,2.0,3.0,3.0,3.0,2.0,...,1.5,2.0,2.0,3.0,2.5,3.0,1.5,2.0,3.0,1.5
3,Welder_4,3.0,2.5,2.0,2.0,2.0,3.0,3.0,3.0,2.0,...,1.5,2.0,2.0,3.0,2.5,3.0,1.5,2.0,3.0,1.5
4,Welder_5,3.0,2.5,2.0,2.0,2.0,3.0,3.0,3.0,2.0,...,1.5,2.0,2.0,3.0,2.5,3.0,1.5,2.0,3.0,1.5


In [55]:
df_data.head()

,Welder_ID,Safety_Rating,Speed_Rating,SMAW_Proficient,GMAW_Proficient,FCAW_Proficient,GTAW_Proficient,Experience_10_Years
0,1,3,4,0,1,1,0,1
1,2,4,1,0,1,1,0,0
2,3,1,2,1,0,0,1,0
3,4,4,2,0,1,0,0,0
4,5,2,3,1,1,1,0,1


#### Decision variables in this binary optimization model:
#### Each binary variable x_i (for i = 1, 2, ..., 144) represents whether a particular welder (applicant i) is hired.
#### If x_i = 1, then welder i is selected for the team; if x_i = 0, then they are not hired.
####
#### There are 144 welders in total, so we have 144 decision variables.


### b

In [56]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ------------------------------
# Data Import
# ------------------------------
# Load welder data and speed data
df_data = pd.read_csv('/Users/Sam/Downloads/welders_data.csv')
df_speed = pd.read_csv('/Users/Sam/Downloads/welders_speed_data.csv')
# We assume df_data columns include:
# "Welder_ID", "Safety_Rating", "Speed_Rating", 
# "SMAW_Proficient", "GMAW_Proficient", "FCAW_Proficient", "GTAW_Proficient", "Experience_10_Years"
# For this model, we primarily use df_data.

# Create dictionary for easier lookup; assume Welder_ID goes from 1 to 144.
welders = df_data.set_index('Welder_ID').to_dict(orient='index')
welder_ids = list(welders.keys())

# ------------------------------
# Create Sets for Constraints
# ------------------------------
# 1. Set SG: Welders proficient in both SMAW and GMAW.
SG = [i for i in welder_ids if (welders[i]['SMAW_Proficient'] == 1 and welders[i]['GMAW_Proficient'] == 1)]

# 2. Sets for each individual welding technique.
S = [i for i in welder_ids if welders[i]['SMAW_Proficient'] == 1]
G = [i for i in welder_ids if welders[i]['GMAW_Proficient'] == 1]
F = [i for i in welder_ids if welders[i]['FCAW_Proficient'] == 1]
T = [i for i in welder_ids if welders[i]['GTAW_Proficient'] == 1]

# 3. Set L: Welders with speed rating = 1.0 and safety rating of 3 or 4.
L = [i for i in welder_ids if (welders[i]['Speed_Rating'] == 1.0 and welders[i]['Safety_Rating'] in [3, 4])]

# 4. Sets for hiring range constraints.
# Set A: Welders with IDs in 70-100.
A = [i for i in welder_ids if 70 <= i <= 100]
# Set B: Welders with IDs in 101-130.
B = [i for i in welder_ids if 101 <= i <= 130]

# ------------------------------
# Build the Gurobi Model: Binary Model
# ------------------------------
binary_model = Model("Welders_Hiring_Binary")

# Create binary decision variables x_i: 1 if welder i is hired, 0 otherwise.
x = {}
for i in welder_ids:
    x[i] = binary_model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
binary_model.update()

# Set the objective: Maximize total speed rating of the selected team.
# (Since team size is fixed at 16, maximizing the sum is equivalent to maximizing the average.)
binary_model.setObjective(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids), GRB.MAXIMIZE)

# ------------------------------
# Add the 9 Standard Constraints
# ------------------------------

# 1. Team Size Constraint: Exactly 16 welders must be hired.
binary_model.addConstr(quicksum(x[i] for i in welder_ids) == 16, name="TeamSize")

# 2. At Least 50% Proficient in Both SMAW and GMAW: >= 8 welders.
binary_model.addConstr(quicksum(x[i] for i in SG) >= 8, name="Proficient_SMAW_GMAW")

# 3. At Least Two Welders Proficient in Each Technique:
#    - SMAW:
binary_model.addConstr(quicksum(x[i] for i in S) >= 2, name="SMAW_min2")
#    - GMAW:
binary_model.addConstr(quicksum(x[i] for i in G) >= 2, name="GMAW_min2")
#    - FCAW:
binary_model.addConstr(quicksum(x[i] for i in F) >= 2, name="FCAW_min2")
#    - GTAW:
binary_model.addConstr(quicksum(x[i] for i in T) >= 2, name="GTAW_min2")

# 4. At Least 30% with More Than 10 Years of Experience:
# Since 30% of 16 is 4.8, we require at least 5 welders.
binary_model.addConstr(quicksum(welders[i]['Experience_10_Years'] * x[i] for i in welder_ids) >= 5, name="Experience")

# 5. Average Safety Rating >= 3.9:
# Total safety must be at least 3.9 * 16 = 62.4.
binary_model.addConstr(quicksum(welders[i]['Safety_Rating'] * x[i] for i in welder_ids) >= 62.4, name="Safety_Avg")

# 6. Average Speed Rating >= 3.1:
# Total speed must be at least 3.1 * 16 = 49.6.
binary_model.addConstr(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids) >= 49.6, name="Speed_Avg")

# 7. At Least Three Welders with Speed = 1.0 and Safety Rating of 3 or 4.
binary_model.addConstr(quicksum(x[i] for i in L) >= 3, name="LowSpeed_HighSafety")

# 8. Hiring Range Constraint:
# Number hired from IDs in 70-100 must be at least one more than twice the number hired from IDs in 101-130.
binary_model.addConstr(quicksum(x[i] for i in A) >= 1 + 2 * quicksum(x[i] for i in B), name="HiringRange")

# 9. Binary Variable Constraints:
# These are automatically enforced by setting vtype=GRB.BINARY for each x[i].

binary_model.update()

# ------------------------------
# Solve the Binary Model
# ------------------------------
binary_model.optimize()

# Capture and print the optimal objective value for the binary model.
binary_obj_val = binary_model.ObjVal
print("Optimal Objective Value (Binary Model):", binary_obj_val)

# ------------------------------
# Solve the LP Relaxation: Change binary variables to continuous [0,1]
# ------------------------------
lp_model = binary_model.copy()
for v in lp_model.getVars():
    v.vtype = GRB.CONTINUOUS  # Relax the binary requirement to continuous variables
lp_model.update()
lp_model.optimize()

lp_obj_val = lp_model.ObjVal
print("Optimal Objective Value (LP Relaxation):", lp_obj_val)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 144 columns and 905 nonzeros
Model fingerprint: 0x1197f922
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 0 rows and 73 columns
Presolve time: 0.00s
Presolved: 11 rows, 71 columns, 406 nonzeros
Variable types: 0 continuous, 71 integer (69 binary)

Root relaxation: objective 5.000000e+01, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      50.0000000   50.00000  0.00%     -    0s

Explored 1 nodes (15 simplex iterations

#### The binary model's optimal objective value is 50.0.
#### The LP relaxation's optimal objective value is 50.6.


### c

#### The fact that the LP relaxation’s optimal value (50.6) is higher than the binary model’s optimal value (50.0) indicates a nonzero integrality gap. This means that relaxing the binary restrictions leads to a solution that is not feasible for the original integer problem. If the two values were equal, it would imply that the LP relaxation is tight—i.e., its optimal solution is already integer—and solving the LP relaxation alone would yield the optimal solution for the binary program, simplifying the computational process.

### d

In [57]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ------------------------------
# Data Import
# ------------------------------
df_data = pd.read_csv('/Users/Sam/Downloads/welders_data.csv')
df_speed = pd.read_csv('/Users/Sam/Downloads/welders_speed_data.csv')

# Create dictionary for lookup; assume Welder_ID goes from 1 to 144.
welders = df_data.set_index('Welder_ID').to_dict(orient='index')
welder_ids = list(welders.keys())

# ------------------------------
# Create Sets for Constraints (same as before)
# ------------------------------
SG = [i for i in welder_ids if (welders[i]['SMAW_Proficient'] == 1 and welders[i]['GMAW_Proficient'] == 1)]
S  = [i for i in welder_ids if welders[i]['SMAW_Proficient'] == 1]
G  = [i for i in welder_ids if welders[i]['GMAW_Proficient'] == 1]
F  = [i for i in welder_ids if welders[i]['FCAW_Proficient'] == 1]
T  = [i for i in welder_ids if welders[i]['GTAW_Proficient'] == 1]

# Set L is NOT used now since we are removing the constraint that requires welders from L.
# L = [i for i in welder_ids if (welders[i]['Speed_Rating'] == 1.0 and welders[i]['Safety_Rating'] in [3, 4])]

A = [i for i in welder_ids if 70 <= i <= 100]
B = [i for i in welder_ids if 101 <= i <= 130]

# ------------------------------
# Build the Modified Gurobi Binary Model (Without Constraint 7)
# ------------------------------
mod_model = Model("Welders_Hiring_Modified")

# Create binary decision variables x_i for each welder.
x = {}
for i in welder_ids:
    x[i] = mod_model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
mod_model.update()

# Objective: Maximize total speed rating of the team.
mod_model.setObjective(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids), GRB.MAXIMIZE)

# ------------------------------
# Add the Constraints (excluding the LowSpeed_HighSafety constraint)
# ------------------------------

# 1. Team Size Constraint: Exactly 16 welders must be hired.
mod_model.addConstr(quicksum(x[i] for i in welder_ids) == 16, name="TeamSize")

# 2. At Least 50% Proficient in Both SMAW and GMAW: at least 8 welders.
mod_model.addConstr(quicksum(x[i] for i in SG) >= 8, name="Proficient_SMAW_GMAW")

# 3. At Least Two Welders Proficient in Each Technique:
mod_model.addConstr(quicksum(x[i] for i in S) >= 2, name="SMAW_min2")
mod_model.addConstr(quicksum(x[i] for i in G) >= 2, name="GMAW_min2")
mod_model.addConstr(quicksum(x[i] for i in F) >= 2, name="FCAW_min2")
mod_model.addConstr(quicksum(x[i] for i in T) >= 2, name="GTAW_min2")

# 4. At Least 30% with More Than 10 Years of Experience: at least 5 welders.
mod_model.addConstr(quicksum(welders[i]['Experience_10_Years'] * x[i] for i in welder_ids) >= 5, name="Experience")

# 5. Average Safety Rating >= 3.9: total safety >= 62.4.
mod_model.addConstr(quicksum(welders[i]['Safety_Rating'] * x[i] for i in welder_ids) >= 62.4, name="Safety_Avg")

# 6. Average Speed Rating >= 3.1: total speed >= 49.6.
mod_model.addConstr(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids) >= 49.6, name="Speed_Avg")

# 7. [REMOVED]: At Least Three Welders with Speed = 1.0 and Safety in {3,4} constraint is omitted.

# 8. Hiring Range Constraint: Number hired from IDs 70-100 must be at least one more than twice the number hired from IDs 101-130.
mod_model.addConstr(quicksum(x[i] for i in A) >= 1 + 2 * quicksum(x[i] for i in B), name="HiringRange")

# 9. Binary variable constraints are inherent.

mod_model.update()

# ------------------------------
# Solve the Modified Binary Model
# ------------------------------
mod_model.optimize()
mod_obj_val = mod_model.ObjVal
print("Modified Binary Model Objective Value:", mod_obj_val)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 144 columns and 891 nonzeros
Model fingerprint: 0x8d078ff0
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 0 rows and 73 columns
Presolve time: 0.00s
Presolved: 10 rows, 71 columns, 393 nonzeros
Variable types: 0 continuous, 71 integer (69 binary)

Root relaxation: objective 5.500000e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      55.0000000   55.00000  0.00%     -    0s

Explored 1 nodes (11 simplex iterations

#### The new optimal objective function value is 55.0. 
#### After removing the constraint, the binary model’s optimal objective value increased from 50.0 to 55.0. This change occurs because the removed constraint forced the selection of at least three welders with a speed rating of 1.0 (which are low-speed welders) and a safety rating of 3 or 4. By eliminating this constraint, the model is free to select only welders with higher speed ratings, resulting in a higher total (and hence average) speed rating for the team.

### e

#### There are no redundant constraints in the model. Each constraint—ranging from the fixed team size, the requirement for a minimum number of welders proficient in both SMAW and GMAW, the minimum numbers for each individual welding technique, the experience and average rating requirements for safety and speed, to the hiring range condition—imposes a unique and necessary restriction on the selection of welders. None of these constraints can be derived from or are implied by the others, ensuring that all project requirements are independently satisfied.

### f

In [58]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ------------------------------
# Data Import
# ------------------------------
df_data = pd.read_csv('/Users/Sam/Downloads/welders_data.csv')
df_speed = pd.read_csv('/Users/Sam/Downloads/welders_speed_data.csv')

# Create dictionary for easier lookup; assume Welder_ID goes from 1 to 144.
welders = df_data.set_index('Welder_ID').to_dict(orient='index')
welder_ids = list(welders.keys())

# ------------------------------
# Create Sets for Constraints
# ------------------------------
SG = [i for i in welder_ids if (welders[i]['SMAW_Proficient'] == 1 and welders[i]['GMAW_Proficient'] == 1)]
S  = [i for i in welder_ids if welders[i]['SMAW_Proficient'] == 1]
G  = [i for i in welder_ids if welders[i]['GMAW_Proficient'] == 1]
F  = [i for i in welder_ids if welders[i]['FCAW_Proficient'] == 1]
T  = [i for i in welder_ids if welders[i]['GTAW_Proficient'] == 1]
L  = [i for i in welder_ids if (welders[i]['Speed_Rating'] == 1.0 and welders[i]['Safety_Rating'] in [3, 4])]
A  = [i for i in welder_ids if 70 <= i <= 100]
B  = [i for i in welder_ids if 101 <= i <= 130]

# ------------------------------
# Build the Gurobi Binary Model
# ------------------------------
binary_model = Model("Welders_Hiring_Binary")

# Create binary decision variables x_i: 1 if welder i is hired, 0 otherwise.
x = {}
for i in welder_ids:
    x[i] = binary_model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
binary_model.update()

# Set the objective: Maximize total speed rating of the team.
binary_model.setObjective(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids), GRB.MAXIMIZE)

# ------------------------------
# Add the 9 Standard Constraints
# ------------------------------

# 1. Team Size: Exactly 16 welders.
binary_model.addConstr(quicksum(x[i] for i in welder_ids) == 16, name="TeamSize")

# 2. At least 50% proficient in both SMAW and GMAW: >= 8 welders.
binary_model.addConstr(quicksum(x[i] for i in SG) >= 8, name="Proficient_SMAW_GMAW")

# 3. At least two welders for each technique.
binary_model.addConstr(quicksum(x[i] for i in S) >= 2, name="SMAW_min2")
binary_model.addConstr(quicksum(x[i] for i in G) >= 2, name="GMAW_min2")
binary_model.addConstr(quicksum(x[i] for i in F) >= 2, name="FCAW_min2")
binary_model.addConstr(quicksum(x[i] for i in T) >= 2, name="GTAW_min2")

# 4. At least 30% with more than 10 years experience: >= 5 welders.
binary_model.addConstr(quicksum(welders[i]['Experience_10_Years'] * x[i] for i in welder_ids) >= 5, name="Experience")

# 5. Average Safety Rating >= 3.9: total safety >= 62.4.
binary_model.addConstr(quicksum(welders[i]['Safety_Rating'] * x[i] for i in welder_ids) >= 62.4, name="Safety_Avg")

# 6. Average Speed Rating >= 3.1: total speed >= 49.6.
binary_model.addConstr(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids) >= 49.6, name="Speed_Avg")

# 7. At least three welders with speed=1.0 and safety in {3,4}.
binary_model.addConstr(quicksum(x[i] for i in L) >= 3, name="LowSpeed_HighSafety")

# 8. Hiring Range: Welders from IDs 70-100 must be at least 1+2*(welders from IDs 101-130).
binary_model.addConstr(quicksum(x[i] for i in A) >= 1 + 2 * quicksum(x[i] for i in B), name="HiringRange")

# 9. Binary variable constraints are enforced by variable type.

binary_model.update()

# ------------------------------
# Configure Solution Pool Parameters to enumerate all optimal solutions.
# ------------------------------
binary_model.setParam(GRB.Param.PoolSearchMode, 2)  # Enable solution pool search.
binary_model.setParam(GRB.Param.PoolSolutions, 1000)  # Set maximum number of solutions to collect.
binary_model.setParam(GRB.Param.PoolGap, 0.0)  # Only accept solutions with optimal objective.

# ------------------------------
# Optimize the model
# ------------------------------
binary_model.optimize()

# Retrieve the number of solutions in the pool that have the optimal objective.
num_solutions = binary_model.SolCount
print("Number of solutions with the optimal objective value:", num_solutions)


Set parameter PoolSearchMode to value 2
Set parameter PoolSolutions to value 1000
Set parameter PoolGap to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
PoolSolutions  1000
PoolSearchMode  2
PoolGap  0

Optimize a model with 11 rows, 144 columns and 905 nonzeros
Model fingerprint: 0x1197f922
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 0 rows and 70 columns
Presolve time: 0.00s
Presolved: 11 rows, 74 columns, 423 nonzeros
Variable types: 0 continuous, 74 integer (74 binary)

Root relaxation: objective 5.000000e+01, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Exp

#### There are 12 solutions have the same optimal objective value as the binary program in part (b).

### g

#### From a managerial standpoint, the biggest drawback of making hiring decisions solely based on an individual’s own qualities is that it ignores team dynamics and potential synergies among team members. While a candidate might excel individually, their skills may not complement those of others, leading to poor collaboration, communication issues, or conflicting work styles. This approach can result in a team that, despite having high-performing individuals, may fail to work effectively together, thereby reducing overall team performance and project success.

### h

In [59]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# ------------------------------
# Data Import
# ------------------------------
df_data = pd.read_csv('/Users/Sam/Downloads/welders_data.csv')
df_speed = pd.read_csv('/Users/Sam/Downloads/welders_speed_data.csv')

# Process df_data: assume Welder_ID goes from 1 to 144.
welders = df_data.set_index('Welder_ID').to_dict(orient='index')
welder_ids = list(welders.keys())

# Process df_speed: set the index to the welder names from the first column.
# We assume that the first column ("Unnamed: 0") contains identifiers like "Welder_1", etc.
V = df_speed.set_index('Unnamed: 0')  # V is a DataFrame with rows and columns as "Welder_1", ..., "Welder_144"

# Map welder_ids (assumed numeric 1...144) to string keys "Welder_i"
def welder_key(i):
    return f"Welder_{i}"

# ------------------------------
# Create Sets for Constraints (same as before)
# ------------------------------
SG = [i for i in welder_ids if (welders[i]['SMAW_Proficient'] == 1 and welders[i]['GMAW_Proficient'] == 1)]
S  = [i for i in welder_ids if welders[i]['SMAW_Proficient'] == 1]
G  = [i for i in welder_ids if welders[i]['GMAW_Proficient'] == 1]
F  = [i for i in welder_ids if welders[i]['FCAW_Proficient'] == 1]
T  = [i for i in welder_ids if welders[i]['GTAW_Proficient'] == 1]
L  = [i for i in welder_ids if (welders[i]['Speed_Rating'] == 1.0 and welders[i]['Safety_Rating'] in [3, 4])]
A  = [i for i in welder_ids if 70 <= i <= 100]
B  = [i for i in welder_ids if 101 <= i <= 130]

# ------------------------------
# Build the Gurobi Model: Nonlinear (Quadratic) Model
# ------------------------------
quad_model = Model("Welders_Hiring_Quadratic")

# Create binary decision variables: x[i] = 1 if welder i is hired, 0 otherwise.
x = {}
for i in welder_ids:
    x[i] = quad_model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
quad_model.update()

# ------------------------------
# Set the new quadratic objective function:
#   max sum_{i} v_{ii} x_i + sum_{i<j} v_{ij} x_i x_j
# where v_{ii} is the speed of welder i and v_{ij} is the pairwise speed interaction.
# ------------------------------
quad_obj = quicksum(V.loc[welder_key(i), welder_key(i)] * x[i] for i in welder_ids)
for i in welder_ids:
    for j in welder_ids:
        if j > i:
            quad_obj += V.loc[welder_key(i), welder_key(j)] * x[i] * x[j]
quad_model.setObjective(quad_obj, GRB.MAXIMIZE)

# ------------------------------
# Add the 9 Standard Constraints
# ------------------------------

# 1. Team Size Constraint: Exactly 16 welders.
quad_model.addConstr(quicksum(x[i] for i in welder_ids) == 16, name="TeamSize")

# 2. At Least 50% Proficient in Both SMAW and GMAW: at least 8 welders.
quad_model.addConstr(quicksum(x[i] for i in SG) >= 8, name="Proficient_SMAW_GMAW")

# 3. At Least Two Welders Proficient in Each Technique.
quad_model.addConstr(quicksum(x[i] for i in S) >= 2, name="SMAW_min2")
quad_model.addConstr(quicksum(x[i] for i in G) >= 2, name="GMAW_min2")
quad_model.addConstr(quicksum(x[i] for i in F) >= 2, name="FCAW_min2")
quad_model.addConstr(quicksum(x[i] for i in T) >= 2, name="GTAW_min2")

# 4. At Least 30% with More Than 10 Years of Experience: at least 5 welders.
quad_model.addConstr(quicksum(welders[i]['Experience_10_Years'] * x[i] for i in welder_ids) >= 5, name="Experience")

# 5. Average Safety Rating >= 3.9: total safety >= 62.4.
quad_model.addConstr(quicksum(welders[i]['Safety_Rating'] * x[i] for i in welder_ids) >= 62.4, name="Safety_Avg")

# 6. Average Speed Rating >= 3.1: total speed >= 49.6.
quad_model.addConstr(quicksum(welders[i]['Speed_Rating'] * x[i] for i in welder_ids) >= 49.6, name="Speed_Avg")

# 7. At Least Three Welders with Speed = 1.0 and Safety in {3,4}.
quad_model.addConstr(quicksum(x[i] for i in L) >= 3, name="LowSpeed_HighSafety")

# 8. Hiring Range Constraint: Welders from IDs 70-100 must be at least 1 + 2*(welders from IDs 101-130).
quad_model.addConstr(quicksum(x[i] for i in A) >= 1 + 2 * quicksum(x[i] for i in B), name="HiringRange")

# 9. Binary Variable Constraints are enforced by variable type.

quad_model.update()

# ------------------------------
# Solve the Quadratic Model
# ------------------------------
quad_model.optimize()

print("Optimal Objective Value (Quadratic Model):", quad_model.ObjVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 144 columns and 905 nonzeros
Model fingerprint: 0x5f60575e
Model has 10296 quadratic objective terms
Variable types: 0 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 4e+00]
  QObjective range [2e+00, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 0 rows and 70 columns
Presolve time: 0.00s
Presolved: 11 rows, 74 columns, 423 nonzeros
Presolved model has 2775 quadratic objective terms
Variable types: 0 continuous, 74 integer (74 binary)

Root relaxation: objective 5.745667e+02, 143 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | I

The new objective function is defined as

$$
\max \quad \sum_{i=1}^{144} v_{ii}\,x_i + \sum_{i=1}^{143}\sum_{j=i+1}^{144} v_{ij}\,x_i x_j,
$$

where \(v_{ii}\) is the estimated speed of welder \(i\) (provided by the specialized AI tool) and \(v_{ij}\) is the additional speed gained (or lost) when welders \(i\) and \(j\) work together. The binary variable \(x_i\) equals 1 if welder \(i\) is hired and 0 otherwise.

After optimally solving the corresponding nonlinear (quadratic) optimization problem with the nine standard constraints, the optimal objective value obtained is **442.0**.


### i 

Yes, the nonlinear binary program can be reformulated as a linear program by linearizing the quadratic terms. One common approach is to introduce new binary variables \(z_{ij}\) for each pair \((i,j)\) with \(i < j\), defined as

$$
z_{ij} = x_i x_j,
$$

where \(x_i, x_j \in \{0,1\}\). To enforce this relationship, the following linear constraints are added for each pair \((i,j)\):

$$
z_{ij} \le x_i,
$$

$$
z_{ij} \le x_j,
$$

$$
z_{ij} \ge x_i + x_j - 1.
$$

Then, the quadratic objective function

$$
\max \quad \sum_{i=1}^{144} v_{ii}\,x_i + \sum_{i=1}^{143}\sum_{j=i+1}^{144} v_{ij}\,x_i x_j
$$

can be reformulated as the linear objective

$$
\max \quad \sum_{i=1}^{144} v_{ii}\,x_i + \sum_{i=1}^{143}\sum_{j=i+1}^{144} v_{ij}\,z_{ij}.
$$

This transformation yields an equivalent mixed-integer linear program with additional variables \(z_{ij}\) and the accompanying constraints, while keeping all other constraints unchanged.


### j

#### Even though incorporating pairwise interactions yields a more realistic model of team dynamics than considering individuals in isolation, it still only captures interactions between two welders at a time. For a team of 16 welders, this approach overlooks the complexities of higher-order interactions among three or more team members. Moreover, modeling all possible pairwise combinations among 144 candidates significantly increases the problem’s size and computational complexity, which can make the optimization intractable.

### k

The model would require one binary variable for each unique group of 16 welders selected from 144, which is given by the binomial coefficient

$$
\binom{144}{16} = 687,917,389,635,036,844,569.
$$

This is an astronomically large number of decision variables, making the model computationally intractable to solve optimally with current methods. Although this formulation captures all possible group interactions, the combinatorial explosion leads to extreme complexity and scalability issues, rendering it impractical for real-world hiring decisions.
